In [1]:
import torch
from torch import nn
import dltools

batch_size = 64
train_iter,test_iter,vocab = dltools.load_data_imdb(batch_size)

In [ ]:
class BiRNN(nn.Module):
    def __init__(self,vocab_size,embed_size,num_hiddens,num_layers,**kwargs):
        super(BiRNN,self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.encoder = nn.LSTM(embed_size,num_hiddens,num_layers=num_layers,bidirectional=True)
        self.decoder = nn.Linear(4 * num_hiddens,2)

    def forward(self,inputs):
        embeddings = self.embedding(inputs,T)
        outputs,_ = self.encoder(embeddings)
        encoding = torch.cat((outputs[0],outputs[-1]),dim=1)
        outs = self.decoder(encoding)
        return outs
    

In [ ]:
embed_size,num_hiddens,num_layers = 100,100,2
device = dltools.try_all_gpus()
net = BiRNN(len(vocab),embed_size,num_hiddens,num_layers)

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.LSTM:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])
net.apply(init_weights)

In [ ]:
glove_embedding = dltools.TokenEmbedding('glove.6b.100d')

In [ ]:
embeds = glove_embedding[vocab.idx_to_token]
embeds.shape

In [ ]:
net.embedding.weight.data.copy_(embeds)
net.embedding.weight.requires_grad = False

In [ ]:
lr,nums_epochs = 0.01,5
trainer = torch.optim.Adam(net.parameters(),lr=lr)
loss = nn.CrossEntropyLoss(reduction="none")
dltools.train_ch13(net,train_iter,test_iter,loss,trainer,nums_epochs,device)

In [ ]:
def predict_sentimenet(net,vocab,sequence):
    sequence = torch.tensor(vocab[sequence.split()],device=device)
    label = torch.argmax(net(sequence.reshape(1,-1)),dim=1)
    return 'pos' if label == 1 else 'nag'


In [ ]:
predict_sentimenet(net,vocab,'This moive is bad')